**Name:** Clement Canel  

My code implements a digital version of Mancala. It supports interactive gameplay where two players can compete against each other, an AI, or a random opponent. The AI strategies include Minimax and Alpha-Beta pruning

### Libraries and Frameworks Used
I used `random` to generate moves for the random opponent and `copy` to simulate game states during the Minimax and Alpha-Beta algorithms

- **Interactive Gameplay:** Two players can compete against each other or an AI, with options for a random or strategic AI.
- **Flexible Board Configuration:** Players can adjust the number of pits and initial stones. The rules are based on a simplified Mancala version.
- **Game Logic:** Handles move validation, stone redistribution, capturing, and winner calculation.
- **AI Player:**  
  - **Minimax:** Explores moves up to a depth to find the best outcome.  
  - **Alpha-Beta Pruning:** Speeds up Minimax by skipping irrelevant branches.  

In [ ]:
import random  
import copy 

random.seed(42)

class MancalaGame:
    def __init__(self, pits_per_side=6, stones_per_pit=4):
        """
        The constructor initializes the Mancala game with the following instance variables:
        
        - pits_per_side: The number of pits each player has.
        - stones_per_pit: The number of stones each pit starts with.
        - board: Represents the current state of the Mancala board.
        - current_player: Tracks whose turn it is (1 for Player 1, 2 for Player 2).
        - player_pits: Maps each player to the indices of their pits on the board.
        - player_mancala: Maps each player to the index of their Mancala on the board.
        """
        self.pits_per_side = pits_per_side
        self.total_pits = (pits_per_side + 1) * 2
        
        self.board = [stones_per_pit] * self.total_pits
        
        self.current_player = 1

        self.player_pits = {
            1: list(range(0, pits_per_side)),  # Player 1 pits
            2: list(range(pits_per_side + 1, 2 * pits_per_side + 1))  # Player 2 pits
        }
        self.player_mancala = {
            1: pits_per_side,  # Player 1 Mancala index
            2: 2 * pits_per_side + 1  # Player 2 Mancala index
        }

        # Set the Mancalas to zero stones initially
        self.board[self.player_mancala[1]] = 0
        self.board[self.player_mancala[2]] = 0
        
    def is_valid_move(self, pit_number):
        """
        Check if a move is valid for the current player.
        A move is valid if:
        - The pit number is within range (1 to pits_per_side).
        - The chosen pit is not empty.
        """
        if pit_number < 1 or pit_number > self.pits_per_side:
            return False
        pit_index = self.player_pits[self.current_player][pit_number - 1]
        return self.board[pit_index] > 0  # Pit must have stones
        
    def display_board(self):
        """
        Displays the current state of the board in a readable format.
        It shows:
        - Player 2's pits.
        - The stone counts in both Mancalas.
        - Player 1's pits.
        - The current player's turn.
        """
        print('==============================')
        print('          CURRENT BOARD        ')
        print('==============================')
        print(f'   P2 Side:  ', end='')
        print('  '.join(f'{self.board[i]:2}' for i in reversed(self.player_pits[2])))
        print(f'    P2 Mancala: {self.board[self.player_mancala[2]]}')
        print('   ===========================   ')
        print(f'    P1 Mancala: {self.board[self.player_mancala[1]]}')
        print(f'   P1 Side:  ', end='')
        print('  '.join(f'{self.board[i]:2}' for i in self.player_pits[1]))
        print(f"\n{'*' * 30}")
        print(f"   It's Player {self.current_player}'s turn!")
        print(f"{'*' * 30}")

    def make_move(self, pit_number):
        """
        Execute a move for the current player.
        - Distribute stones from the chosen pit counterclockwise.
        - Handle special rules such as captures and ending in Mancala.
        """
        if not self.is_valid_move(pit_number):
            print('Invalid move\n')
            return False

        # Pick up stones and clear the chosen pit
        pit_index = self.player_pits[self.current_player][pit_number - 1]
        stones_to_distribute = self.board[pit_index]
        self.board[pit_index] = 0
        position = pit_index

        # Distribute stones
        while stones_to_distribute > 0:
            position = (position + 1) % self.total_pits 
            if position == self.player_mancala[3 - self.current_player]:  
                continue
            self.board[position] += 1
            stones_to_distribute -= 1

        if position in self.player_pits[self.current_player] and self.board[position] == 1:
            opposite_pit = (2 * self.pits_per_side) - position  # Find the opposite pit
            if self.board[opposite_pit] > 0:
                self.board[self.player_mancala[self.current_player]] += self.board[position] + self.board[opposite_pit]
                self.board[position] = 0
                self.board[opposite_pit] = 0

        self.current_player = 3 - self.current_player
        return True

    def check_game_end(self):
        """
        Check if the game has ended.
        The game ends if either player's pits are empty.
        Collect remaining stones from non-empty pits into the respective Mancalas.
        """
        p1_empty = all(self.board[i] == 0 for i in self.player_pits[1])
        p2_empty = all(self.board[i] == 0 for i in self.player_pits[2])

        if p1_empty or p2_empty:
            # Collect remaining stones into Mancalas
            for player in [1, 2]:
                pits = self.player_pits[player]
                mancala = self.player_mancala[player]
                self.board[mancala] += sum(self.board[i] for i in pits)
                for i in pits:
                    self.board[i] = 0

            # Display final board and scores
            self.display_board()
            score_p1 = self.board[self.player_mancala[1]]
            score_p2 = self.board[self.player_mancala[2]]
            print(f'Final Score - Player 1: {score_p1}, Player 2: {score_p2}')
            if score_p1 > score_p2:
                print('Player 1 wins')
            elif score_p2 > score_p1:
                print('Player 2 wins')
            else:
                print('The game is a tie')
            return True
        return False

    def minimax(self, depth, alpha, beta, maximizing_player, player=None):
        """
        Implement the Minimax algorithm with Alpha-Beta pruning.
        - Explore possible moves and evaluate the best outcome.
        - Prune branches that cannot improve the outcome.
        """
        if player is None:
            player = self.current_player
        # Base case
        if depth == 0 or self.check_game_end():
            return self.evaluate_board(player), None
        
        valid_moves = [pit for pit in range(1, self.pits_per_side + 1) if self.is_valid_move(pit)]
        
        # If it's the maximizing player's turn
        if maximizing_player:
            max_eval = float('-inf')  # Initialize the best evaluation score to negative infinity
            best_move = None 
            
            # Iterate through all valid moves
            for move in valid_moves:
                
                simulated_game = copy.deepcopy(self)
                simulated_game.make_move(move) # Execute the move in the simulated game
                
                eval_score, _ = simulated_game.minimax(depth - 1, alpha, beta, False, player)
                
                # Update the best score and move if the current score is better
                if eval_score > max_eval:
                    max_eval = eval_score
                    best_move = move
                    
                # Update alpha (best score for the maximizer)
                alpha = max(alpha, eval_score)
                
                # Prune the branch if beta (minimizer's best score) is less than or equal to alpha
                if beta <= alpha:
                    break 
            return max_eval, best_move
        else:
            min_eval = float('inf')  # Initialize the best evaluation score to positive infinity
            best_move = None # Placeholder for the best move
            
            # Iterate through all valid moves
            for move in valid_moves:
                # Simulate the move by creating a deep copy of the game state
                simulated_game = copy.deepcopy(self)
                simulated_game.make_move(move) # Execute the move in the simulated game
                # Recursively call minimax for the maximizing player
                eval_score, _ = simulated_game.minimax(depth - 1, alpha, beta, True, player)
                
                if eval_score < min_eval:
                    min_eval = eval_score
                    best_move = move
                beta = min(beta, eval_score)
                
                if beta <= alpha:
                    break 
            return min_eval, best_move

    def ai_select_move(self, search_depth):
        """
        Use the Minimax algorithm to determine the best move for the AI.
        Parameters:
        - search_depth: How far to look ahead in the game tree.
        """
        _, chosen_move = self.minimax(search_depth, float('-inf'), float('inf'), True, self.current_player)
        return chosen_move
    
    def evaluate_board(self, player):
        """
        Evaluate the board for the given player.
        Returns the difference in stones between the player's Mancala and the opponent's Mancala.
        """
        return self.board[self.player_mancala[player]] - self.board[self.player_mancala[3 - player]]

# Play the game interactively
game = MancalaGame()
game.display_board()
ai_depth = 3 

# Main game loop
while not game.check_game_end():
    print("-" * 40)  
    if game.current_player == 1:
        valid_move = False
        while not valid_move:
            try:
                player_move = int(input("Player 1, choose a pit (1-6): "))
                valid_move = game.make_move(player_move)
            except ValueError:
                print("Invalid Please enter a number between 1 and 6.")
#    else:
#        # Random player's turn
#        print("Random player is making a move...")
#        random_move_choice = random_move(game)  
#        if random_move_choice:
#            game.make_move(random_move_choice)
#            print(f"Random player chooses pit {random_move_choice}.\n")
#        else:
#            print("No valid moves left for the random player.")
    else:
        # AI's turn
        print("AI MAKING MOVE...")
        ai_move = game.ai_select_move(ai_depth)
        game.make_move(ai_move)
        print(f"AI chooses the pit {ai_move}.\n")
    game.display_board()

print("Game Over!!!")


          CURRENT BOARD        
   P2 Side:   4   4   4   4   4   4
    P2 Mancala: 0
    P1 Mancala: 0
   P1 Side:   4   4   4   4   4   4

******************************
   It's Player 1's turn!
******************************
----------------------------------------


In [4]:
def random_move(game):
    """
    This function selects a random valid move for the current player.
    
    - Parameters:
        - Game: An instance of the MancalaGame.
    - Returns:
        - A random pit number (1-6) that represents a valid move for the player.
        - Returns `None` if there are no valid moves available.
    """
    # Collect all valid pits the player can choose from
    valid_moves = [pit for pit in range(1, game.pits_per_side + 1) if game.is_valid_move(pit)]
    return random.choice(valid_moves) if valid_moves else None


def simulate_random_vs_random(num_games):
    """
    Simulates a series of games where both players make random moves.
    
    - Parameters:
        - num_games: The number of games to simulate.
    - Returns:
        - p1_wins: The number of games Player 1 wins.
        - p2_wins: The number of games Player 2 wins.
        - avg_moves`: The average number of moves per game across all simulations.
    """
    p1_wins, p2_wins, total_moves = 0, 0, 0  
    
    for _ in range(num_games):
        game = MancalaGame()  # Start a new game
        moves = 0  
        
        # Play until the game ends
        while not game.check_game_end():
            move = random_move(game)  
            if move:
                game.make_move(move)  # Execute the move
                moves += 1  

        total_moves += moves 
        
        if game.board[game.player_mancala[1]] > game.board[game.player_mancala[2]]:
            p1_wins += 1
        else:
            p2_wins += 1

    # Calculate the average number of moves per game
    avg_moves = total_moves / num_games
    return p1_wins, p2_wins, avg_moves


def simulate_ai_vs_random(ai_function, depth, num_games):
    """
    Simulates a series of games between an AI player and a random-move player.

    - Parameters:
        - ai_function: A function that determines the AI's move.
        - depth: How deep the AI searches using the Minimax algorithm.
        - num_games: The number of games to simulate.
    - Returns:
        - ai_wins: The number of games the AI wins.
        - random_wins: The number of games the random player wins.
        - avg_moves: The average number of moves per game across all simulations.
    """
    ai_wins, random_wins, total_moves = 0, 0, 0 
    
    # Loop to simulate multiple games
    for _ in range(num_games):
        game = MancalaGame()  
        moves = 0 
        
        while not game.check_game_end():
            if game.current_player == 1:  # AI's turn
                ai_move = ai_function(game, depth)  
                game.make_move(ai_move)  
            else: 
                move = random_move(game) 
                if move:
                    game.make_move(move)  

            moves += 1  # Count the move

        total_moves += moves 
        
        if game.board[game.player_mancala[1]] > game.board[game.player_mancala[2]]:
            ai_wins += 1
        else:
            random_wins += 1

    # Calculate the average number of moves per game
    avg_moves = total_moves / num_games
    return ai_wins, random_wins, avg_moves


print("Simulating Contest Between Two Random Players (100 games)...")
p1_wins, p2_wins, avg_moves = simulate_random_vs_random(100)
print(f"Player 1 Victories: {p1_wins}, Player 2 Victories: {p2_wins}, Average Moves: {avg_moves:.2f}")
print("\nSimulating AI vs Random Player (100 games)...")
ai_victories, random_victories, avg_moves = simulate_ai_vs_random(lambda g, d: g.ai_select_move(d), 3, 100)
print(f"AI Victories: {ai_victories}, Random Player Victories: {random_victories}, Average Moves: {avg_moves:.2f}")


Simulating Contest Between Two Random Players (100 games)...
          CURRENT BOARD        
   P2 Side:   0   0   0   0   0   0
    P2 Mancala: 17
    P1 Mancala: 31
   P1 Side:   0   0   0   0   0   0

******************************
   It's Player 1's turn!
******************************
Final Score - Player 1: 31, Player 2: 17
Player 1 wins
          CURRENT BOARD        
   P2 Side:   0   0   0   0   0   0
    P2 Mancala: 24
    P1 Mancala: 24
   P1 Side:   0   0   0   0   0   0

******************************
   It's Player 1's turn!
******************************
Final Score - Player 1: 24, Player 2: 24
The game is a tie
          CURRENT BOARD        
   P2 Side:   0   0   0   0   0   0
    P2 Mancala: 17
    P1 Mancala: 31
   P1 Side:   0   0   0   0   0   0

******************************
   It's Player 2's turn!
******************************
Final Score - Player 1: 31, Player 2: 17
Player 1 wins
          CURRENT BOARD        
   P2 Side:   0   0   0   0   0   0
    P2 Manca

In [5]:
import time  
def simulate_alpha_beta_vs_random(ai_function, depth, num_games):
    """
    Simulates a series of games between an AI using the Alpha-Beta algorithm and a random player.
    
    Parameters:
    - ai_function: The AI's strategy function, which selects moves using Alpha-Beta pruning.
    - depth: How far ahead the AI should think (depth of search tree).
    - num_games: How many games to play in the simulation.
    
    Returns:
    - ai_wins: How many games the AI won.
    - random_wins: How many games the random player won.
    - avg_moves: The average number of moves made in each game.
    - avg_time_per_game: The average time it took to complete one game.
    """
    ai_wins = 0
    random_wins = 0
    total_moves = 0
    total_time = 0

    for _ in range(num_games):
        # Start a new game
        game = MancalaGame()
        moves = 0 
        start_time = time.time()  

        while not game.check_game_end():
            if game.current_player == 1: 
                ai_move = ai_function(game, depth)  # Let the AI pick its move
                game.make_move(ai_move)
            else: 
                move = random_move(game)  # Pick a random move
                if move:
                    game.make_move(move)
            moves += 1  # Increment the move counter

        total_time += time.time() - start_time
        total_moves += moves

        # Check who won
        if game.board[game.player_mancala[1]] > game.board[game.player_mancala[2]]:
            ai_wins += 1  # AI scored higher, so it wins
        else:
            random_wins += 1 

    # Calculate averages
    avg_moves = total_moves / num_games
    avg_time_per_game = total_time / num_games

    return ai_wins, random_wins, avg_moves, avg_time_per_game


print("Starting the simulation: Alpha-Beta AI vs Random Player!")
print("Running 100 games with AI thinking up to depth 5...\n")
ai_wins, random_wins, avg_moves, avg_time_per_game = simulate_alpha_beta_vs_random(
    lambda g, d: g.ai_select_move(d), 5, 100
)
print(f"Results after 100 games:")
print(f"  - AI Wins: {ai_wins}")
print(f"  - Random Player Wins: {random_wins}")
print(f"  - Average Moves Per Game: {avg_moves:.2f}")
print(f"  - Average Time Per Game: {avg_time_per_game:.2f} seconds")

Starting the simulation: Alpha-Beta AI vs Random Player!
Running 100 games with AI thinking up to depth 5...

          CURRENT BOARD        
   P2 Side:   0   0   0   0   0   0
    P2 Mancala: 15
    P1 Mancala: 33
   P1 Side:   0   0   0   0   0   0

******************************
   It's Player 1's turn!
******************************
Final Score - Player 1: 33, Player 2: 15
Player 1 wins
          CURRENT BOARD        
   P2 Side:   0   0   0   0   0   0
    P2 Mancala: 15
    P1 Mancala: 33
   P1 Side:   0   0   0   0   0   0

******************************
   It's Player 1's turn!
******************************
Final Score - Player 1: 33, Player 2: 15
Player 1 wins
          CURRENT BOARD        
   P2 Side:   0   0   0   0   0   0
    P2 Mancala: 15
    P1 Mancala: 33
   P1 Side:   0   0   0   0   0   0

******************************
   It's Player 1's turn!
******************************
Final Score - Player 1: 33, Player 2: 15
Player 1 wins
          CURRENT BOARD        
   